# Deep Neural Network (manual)

Manual implementation of a multilayer NN (Deep NN)

No gradient descent loop. Just code for one iteration over all examples. 

In [1]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
from testCases import *

np.random.seed(1)

### Initialization

- 2-layer case: The model's structure is: [LINEAR -> RELU] -> [LINEAR -> SIGMOID]
- L-layer case: [LINEAR -> RELU] × (L-1) -> [LINEAR -> SIGMOID]

**Weight/bias matrix sizes:**

Each input goes to each neuron in first layer

--> W1 size is L1_neuron_count x input_size

--> W2 size is L2_neuron_coutn x L1_neuron_count..etc

In [2]:
# 2-layer case
def initialize_parameters(n_x, n_h, n_y):
    """
    Argument:
    n_x -- size of the input layer
    n_h -- size of the hidden layer
    n_y -- size of the output layer
    
    Returns:
    parameters -- python dictionary containing your parameters:
                    W1 -- weight matrix of shape (n_h, n_x)
                    b1 -- bias vector of shape (n_h, 1)
                    W2 -- weight matrix of shape (n_y, n_h)
                    b2 -- bias vector of shape (n_y, 1)
    """
    
    np.random.seed(1)
    # numpy randn gives random numbers 
    # from the normal (gaussian) distribution
    W1 = np.random.randn(n_h, n_x) * 0.01
    b1 = np.zeros((n_h, 1))
    W2 = np.random.randn(n_y, n_h) * 0.01
    b2 = np.zeros((n_y, 1))   
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

# L-layer case
def initialize_parameters_deep(layer_dims):
    """
    Arguments:
    layer_dims -- python array (list) containing the dimensions of each layer in our network
    
    Returns:
    parameters -- python dictionary containing your parameters "W1", "b1", ..., "WL", "bL":
                    Wl -- weight matrix of shape (layer_dims[l], layer_dims[l-1])
                    bl -- bias vector of shape (layer_dims[l], 1)
    """
    
    np.random.seed(3)
    parameters = {}
    L = len(layer_dims) # number of layers in the network

    for l in range(1, L):
        
        parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) * 0.01
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))
        
    return parameters

In [3]:
#2-layer case test
print('# 2-layer case test')
parameters = initialize_parameters(3,2,1)

print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

# L-layer case test
print('\n# L-layer case test')
parameters = initialize_parameters_deep([5,4,3])

print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

# 2-layer case test
W1 = [[ 0.01624345 -0.00611756 -0.00528172]
 [-0.01072969  0.00865408 -0.02301539]]
b1 = [[0.]
 [0.]]
W2 = [[ 0.01744812 -0.00761207]]
b2 = [[0.]]

# L-layer case test
W1 = [[ 0.01788628  0.0043651   0.00096497 -0.01863493 -0.00277388]
 [-0.00354759 -0.00082741 -0.00627001 -0.00043818 -0.00477218]
 [-0.01313865  0.00884622  0.00881318  0.01709573  0.00050034]
 [-0.00404677 -0.0054536  -0.01546477  0.00982367 -0.01101068]]
b1 = [[0.]
 [0.]
 [0.]
 [0.]]
W2 = [[-0.01185047 -0.0020565   0.01486148  0.00236716]
 [-0.01023785 -0.00712993  0.00625245 -0.00160513]
 [-0.00768836 -0.00230031  0.00745056  0.01976111]]
b2 = [[0.]
 [0.]
 [0.]]


### Forward propagation

In [4]:
# Activation functions
def sigmoid(Z):
    """
    sigmoid activation
    
    Arguments:
    Z -- numpy array of any shape
    
    Returns:
    A -- output of sigmoid(z), same shape as Z
    """
    
    A = 1/(1+np.exp(-Z))
    assert(A.shape == Z.shape)   
    
    return A

def relu(Z):
    """
    RELU function.

    Arguments:
    Z -- Output of the linear layer, of any shape

    Returns:
    A -- Post-activation parameter, of the same shape as Z
    """
    
    A = np.maximum(0,Z)
    assert(A.shape == Z.shape)
    
    return A

# Linear
def linear_forward(A, W, b):
    """
    Implement the linear part of a layer's forward propagation.

    Arguments:
    A -- activations from previous layer (or input data): (size of previous layer, number of examples)
    W -- weights matrix: numpy array of shape (size of current layer, size of previous layer)
    b -- bias vector, numpy array of shape (size of the current layer, 1)

    Returns:
    Z -- the input of the activation function, also called pre-activation parameter 
    cache -- a python tuple containing "A", "W" and "b" ; stored for computing the backward pass efficiently
    """
    
    Z = np.matmul(W, A) + b
    
    cache = (A, W, b)
    
    return Z, cache

# Activation
def activation_forward(Z, activation):
    """
    Implement the activation function g

    Arguments:
    Z -- the output of the linear forward function 
    activation -- the activation to be used in this layer, stored as a text string: "sigmoid" or "relu"

    Returns:
    A -- the output of the activation function, also called the post-activation value 
    cache -- a python tuple containing the "activation_cache"
    """
    cache = Z
    if activation == "sigmoid":
        A = sigmoid(Z)
    
    elif activation == "relu":
        A = relu(Z)
    
    return A, cache

# Controller for forward propagation
def forward_propagator(X, parameters):
    """
    Implement forward propagation for the [LINEAR->RELU]*(L-1)->LINEAR->SIGMOID computation
    
    Arguments:
    X -- data, numpy array of shape (input size, number of examples)
    parameters -- output of initialize_parameters_deep(). Can also be used to get the layer count.
    
    Returns:
    AL -- activation value from the output (last) layer
    caches -- list of caches containing:
                every cache of linear_activation_forward() (there are L of them, indexed from 0 to L-1)
    """

    caches = []
    A = X
    L = len(parameters) // 2  # number of layers in the neural network

    # [LINEAR -> RELU]*(L-1). Add "cache" to the "caches" list.
    # The for loop starts at 1 because layer 0 is the input
    for l in range(1, L):
        A_prev = A
        # Linear function
        Z, linear_cache = linear_forward(A_prev, parameters['W' + str(l)], parameters['b' + str(l)])
        # Activation function
        A, activation_cache = activation_forward(Z, "relu")
        cache = (linear_cache, activation_cache)
        caches.append(cache)
        
    # LINEAR -> SIGMOID. Add "cache" to the "caches" list.
    # Linear function
    Z, linear_cache = linear_forward(A, parameters['W' + str(L)], parameters['b' + str(L)])
    # Activation function
    AL, activation_cache = activation_forward(Z, "sigmoid")
    cache = (linear_cache, activation_cache)
    caches.append(cache)
          
    return AL, caches

In [5]:
# Forward prop test

t_X, t_parameters = L_model_forward_test_case_2hidden()
t_AL, t_caches = forward_propagator(t_X, t_parameters)

print("AL = " + str(t_AL))

AL = [[0.03921668 0.70498921 0.19734387 0.04728177]]


### Cost Function

In [6]:
# Logistic Regression cost function (cuz last layer is a sigmoid one)
def compute_cost(AL, Y):
    """
    Arguments:
    AL -- probability vector corresponding to your label predictions, shape (1, number of examples)
    Y -- true "label" vector (for example: containing 0 if non-cat, 1 if cat), shape (1, number of examples)

    Returns:
    cost -- cross-entropy cost
    """
    
    m = Y.shape[1]

    cost = -1/m * np.sum(Y * np.log(AL) + (1 - Y) * np.log(1 - AL))
    
    cost = np.squeeze(cost)

    
    return cost

In [7]:
t_Y, t_AL = compute_cost_test_case()
t_cost = compute_cost(t_AL, t_Y)

print("Cost: " + str(t_cost))

Cost: 0.2797765635793422


### Backward Propagation

For layer $l$, the linear part is: $Z^{[l]} = W^{[l]} A^{[l-1]} + b^{[l]}$ (followed by an activation).

If we have the derivative $dZ^{[l]} = \frac{\partial \mathcal{L} }{\partial Z^{[l]}}$. we can get $(dW^{[l]}, db^{[l]}, dA^{[l-1]})$.

The three outputs $(dW^{[l]}, db^{[l]}, dA^{[l-1]})$ are computed using the input $dZ^{[l]}$.

Here are the formulas you need:
$$ dW^{[l]} = \frac{\partial \mathcal{J} }{\partial W^{[l]}} = \frac{1}{m} dZ^{[l]} A^{[l-1] T} \tag{8}$$
$$ db^{[l]} = \frac{\partial \mathcal{J} }{\partial b^{[l]}} = \frac{1}{m} \sum_{i = 1}^{m} dZ^{[l](i)}\tag{9}$$
$$ dA^{[l-1]} = \frac{\partial \mathcal{L} }{\partial A^{[l-1]}} = W^{[l] T} dZ^{[l]} \tag{10}$$


$A^{[l-1] T}$ is the transpose of $A^{[l-1]}$.

### activation backwards prop
If $g(.)$ is the activation function, 
`sigmoid_backward` and `relu_backward` compute $$dZ^{[l]} = dA^{[l]} * g'(Z^{[l]}). \tag{11}$$ 

In [8]:
# Derivative of the Logistic Regression cost function 
def cost_function_derivative(Y, AL):
    """
    dAL (dJ/dAL : last layer) is the derivative of the logistic regression
    cost function with respect to AL (= y_hat)
    """
    dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL)) # derivative of cost with respect to AL
    return dAL

# g'(Z) depending on activation type 
def activation_function_derivative(Z, activation_type):
    """
    Returns derivative of the activation function of a particular layer l(?)
    gl`(Zl)
    """
    if activation_type == "relu":
        # g(Z) = max(0, Z) --> derivative is 1 when Z >0 and is 0 otherwise
        g_prime = np.where(Z > 0, 1, 0)
        
    elif activation_type == "sigmoid":
        # derivative of sigmoid (s) is  s * (1-s)
        s = sigmoid(Z)
        g_prime = s * (1-s)  
    return g_prime

# dW, db, dA_prev
def get_gradients(dA_l, cache, activation):

    linear_cache, activation_cache = cache
    Z_l = activation_cache
    m = Z_l.shape[1]
    A_prev, W_l, b_l = linear_cache
    g_prime_l = activation_function_derivative(Z_l, activation)    
    dZ_l = dA_l * g_prime_l

    dA_prev_l = np.matmul(W_l.T, dZ_l)
    dW_l = 1/m * np.matmul(dZ_l, A_prev.T)
    db_l = 1/m * np.sum(dZ_l, axis=1, keepdims=True)
    
    return dA_prev_l, dW_l, db_l

# Controller for backwards propagation
def backward_propagator(A_L, Y, caches):
    """
    Backward propagation for a [LINEAR->RELU] * (L-1) -> [LINEAR -> SIGMOID] NN
    
    Arguments:
    A_L -- probability vector, output of the forward propagation (also equals y_hat)
    Y -- true "label" vector (containing 0 if non-cat, 1 if cat)
    caches -- list of caches containing:
                every cache of linear_activation_forward() with "relu" (it's caches[l], for l in range(L-1) i.e l = 0...L-2)
                the cache of linear_activation_forward() with "sigmoid" (it's caches[L-1])
    
    Returns:
    grads -- A dictionary with the gradients
             grads["dA" + str(l)] = ... 
             grads["dW" + str(l)] = ...
             grads["db" + str(l)] = ... 
    """    
    grads = {}
    L = len(caches) # the number of layers
    m = A_L.shape[1]
    Y = Y.reshape(A_L.shape) # after this line, Y is the same shape as AL
    
    # Lth layer (SIGMOID -> LINEAR) gradients. Inputs: "dAL, current_cache". Outputs: "grads["dAL-1"], grads["dWL"], grads["dbL"]
    dA_L = cost_function_derivative(Y, A_L) # derivative of cost with respect to AL
    cache_L = caches[-1] # cache for last layer
    dA_prev_L, dW_L, db_L = get_gradients(dA_L, cache_L, "sigmoid")

    grads["dA" + str(L-1)] = dA_prev_L
    grads["dW" + str(L)] = dW_L
    grads["db" + str(L)] = db_L
    
    # From l=L-1 to 1
    for l in reversed(range(1, L)):
        # lth layer: (RELU -> LINEAR) gradients.
        # Inputs: "grads["dA" + str(l + 1)], current_cache". Outputs: "grads["dA" + str(l)] , grads["dW" + str(l + 1)] , grads["db" + str(l + 1)] 
        
        cache_l = caches[l-1] # list indices are 0~L-1, while layer naming is 1~L (for layer l, index is l-1)
        dA_l = grads["dA" + str(l)]
        
        dA_prev_l, dW_l, db_l = get_gradients(dA_l, cache_l, "relu")
        
        grads["dA" + str(l - 1)] = dA_prev_l
        grads["dW" + str(l)] = dW_l
        grads["db" + str(l)] = db_l

    return grads

In [9]:
## backprop Test
t_AL, t_Y_assess, t_caches = L_model_backward_test_case()
grads = backward_propagator(t_AL, t_Y_assess, t_caches)


print("dA0 = " + str(grads['dA0']))
print("dA1 = " + str(grads['dA1']))
print("dW1 = " + str(grads['dW1']))
print("dW2 = " + str(grads['dW2']))
print("db1 = " + str(grads['db1']))
print("db2 = " + str(grads['db2']))

dA0 = [[ 0.          0.52257901]
 [ 0.         -0.3269206 ]
 [ 0.         -0.32070404]
 [ 0.         -0.74079187]]
dA1 = [[ 0.12913162 -0.44014127]
 [-0.14175655  0.48317296]
 [ 0.01663708 -0.05670698]]
dW1 = [[0.41010002 0.07807203 0.13798444 0.10502167]
 [0.         0.         0.         0.        ]
 [0.05283652 0.01005865 0.01777766 0.0135308 ]]
dW2 = [[-0.39202432 -0.13325855 -0.04601089]]
db1 = [[-0.22007063]
 [ 0.        ]
 [-0.02835349]]
db2 = [[0.15187861]]


### Update the parameters

In [10]:
def update_parameters(params, grads, learning_rate):
    """
    Update parameters using gradient descent
    
    Arguments:
    params -- python dictionary containing your parameters 
    grads -- python dictionary containing your gradients, output of L_model_backward
    
    Returns:
    parameters -- python dictionary containing your updated parameters 
                  parameters["W" + str(l)] = ... 
                  parameters["b" + str(l)] = ...
    """
    parameters = params.copy()
    L = len(parameters) // 2 # number of layers in the neural network

    # Update rule for each parameter
    for l in range(L):
        parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - learning_rate * grads["dW" + str(l+1)]
        parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - learning_rate * grads["db" + str(l+1)]
    return parameters

In [11]:
# Parameter update test
t_parameters, grads = update_parameters_test_case()
t_parameters = update_parameters(t_parameters, grads, 0.1)

print ("W1 = "+ str(t_parameters["W1"]))
print ("b1 = "+ str(t_parameters["b1"]))
print ("W2 = "+ str(t_parameters["W2"]))
print ("b2 = "+ str(t_parameters["b2"]))

W1 = [[-0.59562069 -0.09991781 -2.14584584  1.82662008]
 [-1.76569676 -0.80627147  0.51115557 -1.18258802]
 [-1.0535704  -0.86128581  0.68284052  2.20374577]]
b1 = [[-0.04659241]
 [-1.28888275]
 [ 0.53405496]]
W2 = [[-0.55569196  0.0354055   1.32964895]]
b2 = [[-0.84610769]]
